# Clase `myLogisticRegression`

La clase implementa un modelo de **regresión logística multiclase**, siguiendo exactamente la *generalización multinomial One-vs-All implícita* presentada en la teoría.

El objetivo es estimar la probabilidad de que una observación pertenezca a una de las \(J\) clases posibles, modelando esta probabilidad mediante funciones logísticas y optimizando los parámetros con **descenso del gradiente** sobre la entropía cruzada multiclase.

---

## 1. Modelo Multiclase (One-vs-All implícito)

A diferencia del modelo binario, donde solo existe un vector de pesos \(w\), en el caso multiclase se define un vector para cada clase excepto la última:

$$
w_1, w_2, \dots, w_{J-1}
$$

Cada vector modela la contribución lineal:

$$
z_{ij} = w_j^\top x_i
$$

donde \(x_i\) incluye un primer componente igual a 1 (bias).

---

## 2. Probabilidades de clase

La generalización del Tema 6 utiliza el siguiente esquema:

### Para las clases \(1,\dots,J-1\):

$$
p_{ij} = 
\frac{\exp(w_j^\top x_i)}
{1 + \sum_{m=1}^{J-1}\exp(w_m^\top x_i)}
$$

### Para la última clase \(J\):

$$
p_{iJ} = 1 - \sum_{j=1}^{J-1} p_{ij}
$$

Esta formulación garantiza que:

$$
\sum_{j=1}^{J} p_{ij} = 1
$$

y evita tener que definir un vector de pesos explícito para la última clase.

---

## 3. Función de pérdida: entropía cruzada multiclase

Sea \(Y\) la matriz *one-hot* donde:

- \(Y_{ij} = 1\) si la muestra \(i\) pertenece a la clase \(j\)
- \(0\) en caso contrario

La pérdida es:

$$
E = -\frac{1}{N}\sum_{i=1}^N \sum_{j=1}^J Y_{ij} \log(p_{ij})
$$

---

## 4. Gradiente

El gradiente respecto a cada vector de pesos \(w_j\), para \(j = 1,\dots,J-1\), viene dado por:

$$
\frac{\partial E}{\partial w_j}
=
\frac{1}{N}
\sum_{i=1}^N (p_{ij} - Y_{ij}) x_i
$$

De esta forma, todos los vectores se actualizan simultáneamente en cada iteración.

---

## 5. Regla de actualización

Usando descenso del gradiente con tasa de aprendizaje \(\eta\):

$$
w_j \leftarrow w_j - \eta \, \frac{\partial E}{\partial w_j}
$$

Solo se actualizan los pesos de las primeras \(J-1\) clases, ya que la clase restante está implícita.

---

## 6. Predicción

Para un nuevo vector \(x\):

1. Se calculan las probabilidades generalizadas \(p_{ij}\)
2. Se selecciona la clase con mayor probabilidad:

$$
\hat{y} = \arg\max_j \, p_{ij}
$$

---

## 7. Resumen de características del clasificador

* Modelo multinomial con \(J-1\) vectores de pesos.
* Probabilidades consistentes con la formulación del Tema 6.
* Optimización mediante **descenso del gradiente batch**.
* Compatible con scikit-learn (hereda de `BaseEstimator` y `ClassifierMixin`).
* Implementación totalmente vectorizada.


In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class myLogisticRegression(BaseEstimator, ClassifierMixin):

    def __init__(self, learning_rate=0.01, max_iter=1000, tol=1e-6):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.W_ = None
        self.classes_ = None

    def _softmax_generalized(self, Z):
        """
        Z tiene dimensión (N, J-1).
        Implementa EXACTAMENTE la generalización del Tema 6:

            p_j = exp(z_j) / (1 + sum_m exp(z_m))   para j = 1..J-1
            p_J = 1 - sum_j p_j

        Devuelve probas de dimensión (N, J).
        """
        expZ = np.exp(Z)
        denom = 1 + np.sum(expZ, axis=1, keepdims=True)
        P_small = expZ / denom               # p1..p(J-1)
        p_last = 1 - np.sum(P_small, axis=1, keepdims=True)
        return np.hstack([P_small, p_last])

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).ravel()

        N, k = X.shape
        self.classes_ = np.unique(y)
        J = len(self.classes_)

        # Matriz X con bias
        Xb = np.hstack([np.ones((N, 1)), X])   # (N, k+1)

        # ONE-HOT de y → matriz Y (N, J)
        Y = np.zeros((N, J))
        for idx, c in enumerate(self.classes_):
            Y[y == c, idx] = 1

        # W tendrá dimensión (k+1, J-1)
        rng = np.random.default_rng()
        self.W_ = rng.uniform(-0.01, 0.01, size=(k+1, J-1))

        prev_loss = np.inf

        for _ in range(self.max_iter):

            # Forward → Z = XW (N × (J-1))
            Z = Xb.dot(self.W_)

            # Probabilidades generalizadas (Tema 6, pág. 54)
            P = self._softmax_generalized(Z)  # (N, J)

            # Cross-entropy multiclase
            loss = -np.mean(np.sum(Y * np.log(P + 1e-12), axis=1))

            # Gradiente: dE/dW_j para j=1..J-1
            # Solo actualizamos hasta J-1 (la última clase es implícita)
            error = (P - Y)[:, :J-1]           # (N, J-1)
            grad = Xb.T.dot(error) / N        # (k+1, J-1)

            # Descenso del gradiente
            self.W_ -= self.learning_rate * grad

            # Criterio de parada
            if abs(prev_loss - loss) < self.tol:
                break
            prev_loss = loss

        return self

    def predict(self, X):
        X = np.asarray(X)
        Xb = np.hstack([np.ones((X.shape[0], 1)), X])

        Z = Xb.dot(self.W_)                  # (N, J-1)
        P = self._softmax_generalized(Z)     # (N, J)

        preds_idx = np.argmax(P, axis=1)
        return self.classes_[preds_idx]


In [2]:
import pandas as pd

df = pd.read_csv("data/data_clasificacion.csv")

In [ ]:
X = df.drop(columns=["Class"]).values
y = df["Class"].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)



model = myLogisticRegression(
    learning_rate=0.05,
    max_iter=5000,
    tol=1e-6
)

model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 weighted:", f1_score(y_test, y_pred, average='weighted'))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))



Accuracy: 0.4629917759502112
F1 weighted: 0.4246846544191894
Matriz de confusión:
 [[ 96   0   0  10  13   0   0  19   0  18   0]
 [  0   0  17   0   0  20  39   8  23  46 190]
 [  0   0  71   0   0   6  31   3   8  57 142]
 [ 15   0   0  68   5   1   0   4   0   7   0]
 [ 25   0   0   5  45   0   0   2   0  20   0]
 [  0   0   8   0   0 243  15   0   4  62  30]
 [  1   0  23   4   0  44  73  11  45 121 325]
 [ 16   0   0   5   0   0   0 122   0   1   0]
 [  0   0   2   0   0   1  13   0 253   1 194]
 [ 27   0  25   5   8  43  40   1   5 299 178]
 [ 10   0  71   6  22  16  61   2 142  94 813]]
